In [1]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
from tqdm import tqdm
import librosa
from numba import cuda
import librosa.display
import gc
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 
from sklearn.preprocessing import LabelEncoder
from scipy import signal
from scipy.io import wavfile as wav
from numpy.lib import stride_tricks
from keras.layers import Conv2D, MaxPool2D, Flatten, LSTM, Dense, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
import pandas as pd

Using TensorFlow backend.


In [2]:
from PIL import Image

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')

In [4]:
configuration = tf.compat.v1.ConfigProto()
configuration.gpu_options.per_process_gpu_memory_fraction = 0.5
tf.config.experimental.set_memory_growth(gpus[0],True)
session = tf.compat.v1.Session(config=configuration)

In [5]:
df = pd.read_csv('instruments.csv')
# df.set_index('fname', inplace=True)

# for f in df.index:
#     signal, rate = librosa.load('./cleaned/'+f)
#     df.at[f, 'length'] = signal.shape[0]/rate

# classes = list(np.unique(df.label))
# class_dist = df.groupby(['label'])['length'].mean()

# n_samples = 2 * int(df['length'].sum()/0.1)
# prob_dist = class_dist /class_dist.sum()
# choices = np.random.choice(class_dist.index, p=prob_dist)



# fig, ax = plt.subplots()
# ax.set_title('Class Distribution', y=1.08)
# ax.pie(class_dist, labels=class_dist.index, autopct='%1.1f%%',
#        shadow=False, startangle=90)
# ax.axis('equal')
# plt.show()

In [6]:
classname =df[df.fname == "5388d14d.wav"].label.values

In [7]:
classname[0]

'Saxophone'

In [8]:
def read_audio(conf, pathname, trim_long_data):
    y, sr = librosa.load(pathname, sr=conf.sampling_rate)
    # trim silence
    if 0 < len(y): # workaround: 0 length causes error
        y, _ = librosa.effects.trim(y) # trim, top_db=default(60)
    # make it unified length to conf.samples
    if len(y) > conf.samples: # long enough
        if trim_long_data:
            y = y[0:0+conf.samples]
    else: # pad blank
        padding = conf.samples - len(y)    # add padding at both ends
        offset = padding // 2
        y = np.pad(y, (offset, conf.samples - len(y) - offset), 'constant')
    return y

In [9]:
def audio_to_melspectrogram(conf, audio):
    spectrogram = librosa.feature.melspectrogram(audio, 
                                                 sr=conf.sampling_rate,
                                                 n_mels=conf.n_mels,
                                                 hop_length=conf.hop_length,
                                                 n_fft=conf.n_fft,
                                                 fmin=conf.fmin,
                                                 fmax=conf.fmax)
    spectrogram = librosa.power_to_db(spectrogram)
    spectrogram = spectrogram.astype(np.float32)
    return spectrogram

In [10]:
def read_as_melspectrogram(conf, pathname, trim_long_data, debug_display=False):
    x = read_audio(conf, pathname, trim_long_data)
    mels = audio_to_melspectrogram(conf, x)
    return mels

In [11]:
class conf:
    sampling_rate = 44100
    duration = 2
    hop_length = 347*duration # to make time steps 128
    fmin = 20
    fmax = sampling_rate // 2
    n_mels = 128
    n_fft = n_mels * 20
    samples = sampling_rate * duration

In [13]:
def save_image_from_sound(img_path):
    filename = rename_file(img_path)
    x = read_as_melspectrogram(conf, img_path, trim_long_data=True, debug_display=True)
    
#     plt.imshow(x, interpolation='nearest')
    #plt.savefig('./spectrograms/' + filename)
    
    fig = plt.figure(frameon=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(x, aspect='auto')
    plt.savefig('./spectrograms2/' + filename)
    
    plt.close()
    del x
    gc.collect()

In [14]:
def rename_file(img_name):
    img_name = img_name.split("/")[2]
    img_name = img_name[:-4]
    img_name += ".jpg"
    return img_name

In [15]:

for i, fn in enumerate(os.listdir('./cleaned/')):
    print(i)
    path = './cleaned/' + fn
    save_image_from_sound(path)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27